In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Loading path for mfcc features
MFCC_dataset_path = "/content/drive/MyDrive/Emotion_Recognition/mfcc_train.mat"

In [13]:
# Libraries to be used
import scipy.io
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report

In [14]:
# Loading mfcc data and printing the classes of emotions
mfcc_data = scipy.io.loadmat(MFCC_dataset_path)
emotion_categories = list(mfcc_data.keys())
emotion_categories = [e for e in emotion_categories if not e.startswith("__")]
print("Emotion categories:", emotion_categories)

Emotion categories: ['fear', 'happiness', 'sadness']


In [15]:
# Training Gaussian Mixture Model for every emotion
gmm_models = {}

for emotion in emotion_categories:
    mfcc_list = []
    print(f"Training GMM for emotion: {emotion}")

    for i in range(mfcc_data[emotion].shape[0]):
        mfcc_array = np.array(mfcc_data[emotion][i])
        mfcc_list.append(mfcc_array.T)

    # Making dataset one uniform
    dataset = np.vstack(mfcc_list)

    # Ready-made GMM function
    gmm = GaussianMixture(n_components=64, covariance_type='full', random_state=42, max_iter= 2000, init_params= 'kmeans')
    gmm.fit(dataset)

    gmm_models[emotion] = gmm

print("GMMs trained for each emotion.")


Training GMM for emotion: fear
Training GMM for emotion: happiness
Training GMM for emotion: sadness
GMMs trained for each emotion.


In [16]:
train_predictions = []
for emotion in emotion_categories:
    for i in range(mfcc_data[emotion].shape[0]):
        mfcc_array = np.array(mfcc_data[emotion][i]).T

        log_likelihoods = []
        for model_emotion, model in gmm_models.items():
            log_likelihoods.append(model.score(mfcc_array))

        predicted_emotion = emotion_categories[np.argmax(log_likelihoods)]

        train_predictions.append(predicted_emotion)

In [17]:
train_true_labels = []
for emotion in emotion_categories:
    train_true_labels.extend([emotion] * mfcc_data[emotion].shape[0])

train_accuracy = accuracy_score(train_true_labels, train_predictions)
print(f"Training Accuracy: {train_accuracy}")

train_conf_matrix = confusion_matrix(train_true_labels, train_predictions, labels=emotion_categories)
print("Training Confusion Matrix:")
print(train_conf_matrix)

Training Accuracy: 0.9850746268656716
Training Confusion Matrix:
[[112   0   0]
 [  1 111   0]
 [  2   2 107]]


In [18]:
MFCC_test_dataset_audio = "/content/drive/MyDrive/Emotion_Recognition/mfcc_test.mat"
mfcc_test_data = scipy.io.loadmat(MFCC_test_dataset_audio)

emotion_categories_test = list(mfcc_test_data.keys())
emotion_categories_test = [e for e in emotion_categories_test if not e.startswith("__")]

In [19]:
true_labels = []
predicted_labels = []

for emotion in emotion_categories_test:
    for i in range(mfcc_test_data[emotion].shape[0]):
        mfcc_array = np.array(mfcc_test_data[emotion][i]).T
        log_likelihoods = []
        for model_emotion, model in gmm_models.items():
            log_likelihoods.append(model.score(mfcc_array))

        predicted_emotion = emotion_categories[np.argmax(log_likelihoods)]

        true_labels.append(emotion)
        predicted_labels.append(predicted_emotion)

In [20]:
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision (weighted): {precision}")
print(f"Recall (weighted): {recall}")
print(f"F1-score (weighted): {f1}")

print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, labels=emotion_categories))

conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=emotion_categories)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.6666666666666666
Precision (weighted): 0.5
Recall (weighted): 0.6666666666666666
F1-score (weighted): 0.5555555555555555

Classification Report:
              precision    recall  f1-score   support

        fear       0.50      1.00      0.67         1
   happiness       1.00      1.00      1.00         1
     sadness       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.50      0.67      0.56         3
weighted avg       0.50      0.67      0.56         3

Confusion Matrix:
[[1 0 0]
 [0 1 0]
 [1 0 0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m